### Import thư viện

In [ ]:
import os
import re
import json
import random
import shutil

In [ ]:
import pandas as pd
import translators as ts

### Giữ lại một số lượng file phù hợp

Dữ liệu tải về được lưu trong 2 thư mục cnn và dailymail, do hạn chế về việc lưu trữ data cũng như các đơn vị tính toán trên laptop cá nhân, nhóm chỉ sử dụng 150 000 files dữ liệu được trộn hỗn hợp từ cnn và dailymail:

<em>Vui lòng không chạy code này !</em>

In [ ]:
path_folder = "../../data/" # đường đẫn dễn thư mục chứa file lưu trữ

folder_cnn = f"{path_folder}cnn"
folder_daily = f"{path_folder}dailymail"

In [ ]:
def copy_random_files(source_folder, destination_folder, num_files_to_select):
    all_files = os.listdir(source_folder)
    selected_files = random.sample(all_files, min(num_files_to_select, len(all_files)))
    for file_name in selected_files:
        source_path = os.path.join(source_folder, file_name)
        destination_path = os.path.join(destination_folder, file_name)
        shutil.copy(source_path, destination_path)

number = 150000 # Số lượng file giữ lại.

# Tạo và xác định thư mục đích
destination_folder = '../data_summarize/'
os.makedirs(destination_folder, exist_ok=True)

# Lấy ngẫu nhiên các tệp từ thư mục cnn vài daily
copy_random_files(folder_cnn, destination_folder, number // 2)
copy_random_files(folder_daily, destination_folder, number // 2)

print(f'{number} tệp đã được chọn và sao chép thành công.')

### Thực hiện ghép các đoạn rời rạc và thực hiện lưu lại file dưới dạng json

Do dữ liệu được lưu trữ dưới dạng file STORY, các câu đoạn được trích xuất từ các thẻ HTML nên câu văn bị ngắt nghỉ chưa đúng chỗ, ngoài ra các ý chính (đại diện cho mục tóm tắt) cũng bị ngăn cách bởi các ký tự highlight. 

Cần thực hiện ghép nối.

In [ ]:
data_folder = "../data_summarize/"
files = os.listdir(data_folder)

In [ ]:
for file in files:
    with open(f"{data_folder}{file}", "r+", encoding="utf-8") as f:
        content = f.read()

        # Tìm vị trí các highlight
        highlight_index = content.find('@highlight')
        # Chia nội dung thành 2 phần: nội dung chính và tóm tắt
        news = content[:highlight_index].replace('\n', ' ').strip()
        summary = content[highlight_index + len('@highlight'):]\
                        .replace('\n', ' ').replace('@highlight', '').strip()
        # Loại bỏ các khoảng trắng
        news = re.sub(r'\s+', ' ', news)
        summary = re.sub(r'\s+', ' ', summary)
        json_data = {
            "news": news,
            "summary": summary
        }

        f.seek(0)
        # Chuyển đối tượng Python thành chuỗi JSON và ghi vào tệp tin
        json.dump(json_data, f, ensure_ascii=False, indent=2)
        # Cắt bớt phần còn lại của tệp tin (nếu có)
        f.truncate()

### Thực hiện translate phần summary tạo target cho mô hình translate

Với phần translate, nhóm sử dụng module translators cũng như đã thử nghiệm một số mô hình pretrained, kết quả dịch không đúng nghĩa, sát nghĩa,... dịch thiếu tokens,... nên nhóm thực hiện gán nhãn bằng tay với công cụ hỗ trợ: google dịch, gpt, poe.com, ...

Do nguồn nhân lực và thời gian còn hạn chế, nhóm chỉ gán nhãn 1000 file bất kỳ để sử dụng cho quá tình fine-tuning model translate.

In [ ]:
for file in files[:1000]:
    with open(f"{data_folder}{file}", "r+", encoding="utf-8") as f:
        print(file)
        content = json.load(f)
        translate_summary = ts.translate_text(content["summary"], # sử dụng google để dịch
                                            from_language='en', to_language='vi', 
                                            translator='google')
        content["translate"] = translate_summary
        f.seek(0)
        json.dump(content, f, ensure_ascii=False, indent=2)
        f.truncate()
        shutil.copy(f"{path_folder}{file}",f"../data_translate/{file[:-5]}txt")